Jeju_DataPreproces_03th에서 만든 함수 사용, groupby 별 군집(범주형 파생변수) 제작

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
import gc
import warnings
from scipy.spatial import distance
from math import log2
from math import sqrt
from sklearn.cluster import AgglomerativeClustering
import time
warnings.filterwarnings(action = 'ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/st_all.csv')

In [ ]:
train_df = data[(data['REG_YYMM'] != 202007)&(data['REG_YYMM'] != 202004)]
test_df = data[(data['REG_YYMM'] == 202007)|(data['REG_YYMM'] == 202004)]

In [ ]:
def Jensenshannon_similarity(df,input_list):
  '''
  [함수 사용법]
    REG_YYMM을 input_list 인덱싱 0번 자리(맨앞)에 둘 것
    그 뒤로 기준을 잡고 싶은 변수들을 차례로 넣어 주면 됩니다.
    from scipy.spatial import distance 꼭 할것!
  '''


  make_amt= df.groupby(input_list)['AMT'].sum().reset_index()
  
  '''pivot 만들기'''
  amt_sum = make_amt.groupby(input_list[1:])['AMT'].sum().reset_index(name='AMT_sum')
  make_amt = make_amt.merge(amt_sum,on=input_list[1:],how='left')
  make_amt['AMT_prob'] = make_amt['AMT']/make_amt['AMT_sum']

  # pivot_table을 활용하여 long to wide 작업 실시
  amt_pivot= pd.pivot_table(make_amt,
                            index=input_list[1:],
                            columns='REG_YYMM',
                            values='AMT_prob').reset_index()
  amt_pivot.fillna(0,inplace=True)

  #피벗된 연도별 amt를 사용해 고객간의 유사도를 구해보자.
  amt_array=np.array(amt_pivot.drop(input_list[1:],axis=1))

  js_similarity =[]
  for row_r in range(amt_array.shape[0]):       
    dt_list=[]
    for row_c in range(amt_array.shape[0]):
        #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
        '''
        js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
        sqrt(js_divergence)인 distance.jensenshannon 함수 사용
        '''
        dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
        dt_list.append(dt)
    js_similarity.append(dt_list)

  js_frame =pd.DataFrame(np.array(js_similarity))
  js_similarity = pd.concat([amt_pivot,js_frame],axis=1)

  return js_similarity

In [ ]:
def make_clustering(gp_df,input_list,col_name):
    df = Jensenshannon_similarity(gp_df,input_list)
    df_s = df.iloc[:,-df.shape[0]:] # 클러스터링용 dataframe 생성
    clustering = AgglomerativeClustering(n_clusters=int(df_s.shape[0]/2),
                                     affinity='precomputed',
                                     linkage='average')
    clustering.fit(df_s)
    
    dfd = df.iloc[:,:-df.shape[0]]
    dfd['cluster'] = clustering.labels_
    dfd.rename(columns={'cluster':col_name},inplace=True)

    return dfd.drop(dfd.iloc[:,-16:-1].columns,axis=1)

In [ ]:
# 업종 clustering

STD = make_clustering(train_df,['REG_YYMM','STD_CLSS_NM'],'STD')
SIDO_STD = make_clustering(train_df,['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'],'SIDO_STD')
SIDO_CCG_STD = make_clustering(train_df,['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM'],'SIDO_CCG_STD')

# 소비자 clustering

ASF = make_clustering(train_df,['REG_YYMM','AGE','SEX_CTGO_CD','FLC'],'ASF')
SIDO_ASF = make_clustering(train_df,['REG_YYMM', 'HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF')
SIDO_CCG_ASF = make_clustering(train_df,['REG_YYMM', 'HOM_SIDO_NM', 'HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF')

In [ ]:
for data in[STD,SIDO_STD,SIDO_CCG_STD,ASF,SIDO_ASF,SIDO_CCG_ASF]:
  data.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/%s.csv'%(data.columns[-1]),encoding='utf-8-sig',index=False)